In [85]:
import numpy as np
from typing import List, Dict, Union, Tuple
from tenpy.networks.site import SpinSite
from tenpy.linalg import np_conserved as npc
from tenpy.linalg.np_conserved import Array
from tenpy.networks.mpo import MPO
from tenpy.models.model import MPOModel
from tenpy.models.lattice import Chain

In [86]:
def build_mpo(B_list, d):
    """
    Costruisce un MPO diagonale con blocchi W_i[:,x,x,:] = B_i^x.
    Ogni B_i è un dizionario {x: matrice (Dl,Dr)} e B_list è la lista su tutti i siti.
    """
    N = len(B_list)

    # 1) Definisci i siti TenPy (spazio locale di dimensione d).
    #    SpinSite con S=(d-1)/2 genera spazio di dimensione d, senza simmetrie conservate.
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere un intero positivo.")
    S = (d - 1) / 2.0
     # 1) crea il lattice PRIMA
    base_site = SpinSite(S=S, conserve=None)
    lattice = Chain(L=N, site=base_site)

    # 2) prendi i Site che il lattice userà davvero
    sites = lattice.mps_sites() 
    
    Ws = []
    for i in range(N):
        Bi = B_list[i]  # dizionario {x: matrice (Dl,Dr)}

        # --- controlli di base sul dizionario Bi ---
        if not isinstance(Bi, dict) or len(Bi) == 0:
            raise ValueError(f"Sito {i}: Bi deve essere un dict non vuoto.")
        # devono esistere tutte le chiavi x = 0..d-1
        missing = [x for x in range(d) if x not in Bi]
        if missing:
            raise KeyError(f"Sito {i}: mancano chiavi {missing} in Bi.")

        # 2) Imposta le dimensioni di bond coerenti con la posizione del sito.
        #    Dl = 1 al bordo sinistro, Dr = 1 al bordo destro, altrimenti 2.
        Dl = 1 if i == 0 else 2
        Dr = 1 if i == N - 1 else 2

        # ricava le shape attese dalle matrici in Bi e verifica coerenza
        shapes = {np.shape(M) for M in Bi.values()}
        if len(shapes) != 1:
            raise ValueError(f"Sito {i}: shape incoerenti in Bi: {shapes}")
        Dl_bi, Dr_bi = next(iter(shapes))
        if (Dl_bi, Dr_bi) != (Dl, Dr):
            raise ValueError(
                f"Sito {i}: atteso (Dl,Dr)=({Dl},{Dr}), trovato ({Dl_bi},{Dr_bi})."
            )

        # determina dtype (float o complex) dalle matrici fornite
        # se sono reali resta float, se compare una parte complessa si promuove a complex
        dtype = np.result_type(*[np.array(Bi[x]).dtype for x in range(d)])

        # 3) Alloca il tensore MPO W_i con shape (Dl, d, d, Dr).
        #    Assi TenPy: ['wL','p','p*','wR'] = (bond_sx, ket, bra, bond_dx).
        W = np.zeros((Dl, d, d, Dr), dtype=dtype)

        # 4) Riempie solo la diagonale fisica x==y con le matrici B_i^x fornite.
        for x in range(d):
            Mx = np.asarray(Bi[x], dtype=dtype)
            # broadcast sulla coppia (Dl,Dr) nei posti (wL,wR) e nel blocco fisico (x,x)
            W[:, x, x, :] = Mx

        # 5) Converte in Array TenPy con le etichette richieste.
        W_arr = npc.Array.from_ndarray_trivial(W, labels=['wL', 'p', 'p*', 'wR'])
        Ws.append(W_arr)

    # 6) Assembla l’MPO finale con condizioni al contorno finite (aperte).
    H_MPO = MPO(sites, Ws, bc='finite')

    # 7) Wrappa l’MPO nel modello TenPy usando lo STESSO lattice    
    model = MPOModel(lattice, H_MPO)

    # 8) Restituisce sia il modello che l’MPO
    return model, H_MPO

In [87]:
def build_mps(B_list, d):
    """
    Crea una MPS con A[i][:, x, :] = B_i[x].
    B_list: lista di dict {x -> np.ndarray (Dl, Dr)} per i=0..N-1
    d: dimensione fisica (x = 0..d-1)
    """
    N = len(B_list)
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")

    S = (d - 1) / 2.0
    base_site = SpinSite(S=S, conserve=None)
    lattice = Chain(L=N, site=base_site)
    sites = lattice.mps_sites()

    A = []
    right_dims = []  # per costruire SVs

    for i, Bi in enumerate(B_list):
        if not isinstance(Bi, dict) or len(Bi) == 0:
            raise ValueError(f"sito {i}: Bi deve essere dict non vuoto.")

        shapes = {np.shape(M) for M in Bi.values()}
        if len(shapes) != 1:
            raise ValueError(f"sito {i}: shape incoerenti {shapes}")
        Dl, Dr = next(iter(shapes))
        if i == 0 and Dl != 1:
            raise ValueError(f"sito {i}: Dl deve essere 1, trovato {Dl}")
        if i == N - 1 and Dr != 1:
            raise ValueError(f"sito {i}: Dr deve essere 1, trovato {Dr}")

        dtype = np.result_type(*[np.asarray(Bi.get(x, 0)).dtype for x in range(d)])

        Ai = np.zeros((Dl, d, Dr), dtype=dtype)
        for x in range(d):
            Mx = np.asarray(Bi.get(x, np.zeros((Dl, Dr), dtype=dtype)), dtype=dtype)
            if Mx.shape != (Dl, Dr):
                raise ValueError(f"sito {i}: B[{x}] ha shape {Mx.shape}, atteso {(Dl,Dr)}")
            Ai[:, x, :] = Mx

        A_i = npc.Array.from_ndarray_trivial(Ai, labels=['vL', 'p', 'vR'])
        A.append(A_i)
        right_dims.append(Dr)

    # SVs: N-1 vettori, uno per ogni legame interno; qui li mettiamo tutti a 1
    svs = [np.ones(1, dtype=float)]  # S_0
    svs += [np.ones(right_dims[i], dtype=float) for i in range(N - 1)]  # S_1..S_{N-1}
    svs += [np.ones(1, dtype=float)]  # S_N

    # Costruisci la MPS
    psi = MPS(sites, A, svs, bc='finite', form='A')

    return psi


In [88]:
def build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu):
    pmid = 1 - pd - pu
    if not (0 <= pd <= 1 and 0 <= pu <= 1 and pd + pu <= 1):
        raise ValueError("Probabilità non valide: servono pd, pu >=0 e pd+pu <= 1")

    B_list = []

    # Sito 1: (1,2)
    B1 = {
        0: np.array([[ (S0/(N+1))*d_op*pd,  (S0/(N+1))*d_op*pd - K*pd ]]),
        1: np.array([[ (S0/(N+1))*m_op*pmid, ((S0/(N+1))*m_op - K)*pmid ]]),
        2: np.array([[ (S0/(N+1))*u_op*pu,  (S0/(N+1))*u_op*pu - K*pu ]]),
    }
    B_list.append(B1)

    # Siti 2..N-1: (2,2)  ← QUI MANCANO NEL TUO FILE
    for i in range(2, N):
        Bi = {
            0: np.array([[ d_op*pd,  (S0/(N+1))*d_op*pd - K*pd ],
                         [     pd,                               1 ]]),
            1: np.array([[ m_op*pmid, ((S0/(N+1))*m_op - K)*pmid ],
                         [      pmid,                               1 ]]),
            2: np.array([[ u_op*pu,  (S0/(N+1))*u_op*pu - K*pu ],
                         [    pu,                                 1 ]]),
        }
        B_list.append(Bi)

    # Sito N: (2,1)
    BN = {
        0: np.array([[ d_op*pd ],
                     [     pd ]],),
        1: np.array([[ m_op*pmid ],
                     [      pmid ]]),
        2: np.array([[ u_op*pu ],
                     [     pu ]]),
    }
    B_list.append(BN)
    return B_list

In [89]:
S0, K, N = 1.0, 0.2, 4
d_op, m_op, u_op = 0.9, 1.0, 1.1
pd, pu = 0.4, 0.3   

B_list = build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu)
print(B_list)

# B_list è una lista di dizionari che definisce i blocchi locali dell'MPO.
# - L’indice i (0 ≤ i < N) identifica il sito nella catena.
# - Ogni elemento B_list[i] è un dizionario {x: matrice}, dove x ∈ {0, 1, 2} è l’indice fisico locale.
# - La matrice B_list[i][x] (shape = (D_{i-1}, D_i)) rappresenta il blocco virtuale B_i^x.
# In formula:  B_list[i][x]  ≡  B_i^{x}
# Esempio di accesso:  B_list[0][2] restituisce la matrice B_1^{2}.

[{0: array([[ 0.072, -0.008]]), 1: array([[0.06, 0.  ]]), 2: array([[0.066, 0.006]])}, {0: array([[ 0.36 , -0.008],
       [ 0.4  ,  1.   ]]), 1: array([[0.3, 0. ],
       [0.3, 1. ]]), 2: array([[0.33 , 0.006],
       [0.3  , 1.   ]])}, {0: array([[ 0.36 , -0.008],
       [ 0.4  ,  1.   ]]), 1: array([[0.3, 0. ],
       [0.3, 1. ]]), 2: array([[0.33 , 0.006],
       [0.3  , 1.   ]])}, {0: array([[0.36],
       [0.4 ]]), 1: array([[0.3],
       [0.3]]), 2: array([[0.33],
       [0.3 ]])}]


In [90]:
model, H_MPO = build_mpo(B_list, d=3)
psi = build_mps(B_list, d=3)